# Differential expression and celltype analysis [All Cell]

1. Differential gene expression (DGE) analysis identifies genes that show statistically significant differences in expression levels across distinct cell populations or conditions. This analysis helps in identifying which cell types are most affected by a condition of interest such as a disease, and characterizing their functional signatures.

2. Differential Compositional analysis identifies Quantifies changes in the relative abundances of each cell type across conditions (e.g., case vs. control, time points, treatment groups). Reveals expansions or contractions of specific populations that may not be captured by gene-level analyses alone.

Here, we introduced `omicverse.single.DEG` and `omicverse.single.DCT` to performed these two analysis.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import pertpy as pt
import omicverse as ov
ov.plot_set(font_path='Arial')
!date

🔬 Starting plot initialization...
Using already downloaded Arial font from: /tmp/omicverse_arial.ttf
Registered as: Arial
🧬 Detecting CUDA devices…
✅ [GPU 0] NVIDIA GeForce RTX 2080 Ti
    • Total memory: 10.7 GB
    • Compute capability: 7.5
✅ [GPU 1] NVIDIA GeForce RTX 2080 Ti
    • Total memory: 10.7 GB
    • Compute capability: 7.5

   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

🔖 Version: 1.7.6rc1   📚 Tutorials: https://omicverse.readthedocs.io/
✅ plot_set complete.

2025年 08月 05日 星期二 12:26:22 CST


## Data Preprocess

The data we use in the following example comes from [Haber et al., 2017]. It contains samples from the small intestinal epithelium of mice with different conditions. We first load the raw cell-level data. The dataset contains gene expressions of 9842 cells. They are annotated with their sample identifier (`batch`), the condition of the subjects and the type of each cell (`cell_label`).

In [3]:
adata = pt.dt.haber_2017_regions()
adata

AnnData object with n_obs × n_vars = 9842 × 15215
    obs: 'batch', 'barcode', 'condition', 'cell_label'

For our first example, we want to look at how the `Salmonella` infection influences the cell composition. Therefore, we create a subset of our compositional data that only contains the `healthy` and `Salmonella-infected` samples as a new data modality.

In [4]:
# Select control and salmonella data
adata = adata[
    adata.obs["condition"].isin(["Control", "Salmonella"])
].copy()
print(adata)

AnnData object with n_obs × n_vars = 5010 × 15215
    obs: 'batch', 'barcode', 'condition', 'cell_label'


In [5]:
adata.obs["condition"].unique()

['Control', 'Salmonella']
Categories (2, object): ['Control', 'Salmonella']

## DEG with wilcoxon/t-test

In omicverse, we only need one function `ov.single.DEG` to perfrom all DEG analysis tasks. First, I will introduce the nonparametric Wilcoxon test and the t-test—two widely used methods for differential expression analysis. Due to their high computational efficiency, we applied them in our DEG analysis.

We need to set the `ctrl_group` and `test_group` to perform the analysis. Besides, we also need to define the celltype to be explode. If you set `celltype_group` is None, all the celltype will be calculated.

In [6]:
deg_obj=ov.single.DEG(
    adata,
    condition='condition',
    ctrl_group='Control',
    test_group='Salmonella',
    method='wilcoxon',
)
deg_obj.run(
    celltype_key='cell_label',
    celltype_group=['TA'],
)


✅ Differential expression analysis initialized
📊 DEG analysis using wilcoxon method
📊 Condition: condition, Control group: Control, Test group: Salmonella
📊 Celltype key: cell_label, Celltype group: ['TA']
Total cells: 533 will be used for DEG analysis
normalizing counts per cell
    finished (0:00:00)
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
✅ wilcoxon DEG analysis completed


In [7]:
res_wilcoxon=deg_obj.get_results()
res_wilcoxon.head()

,log2FC,pvalue,padj,qvalue,size,sig,-log(pvalue),-log(qvalue),baseMean
Reg3b,5.958403,2.899264e-58,4.411230e-54,4.411230e-54,0.595840,sig,57.537712,53.355440,1.041882
Reg3g,4.425004,1.576382e-51,1.199233e-47,1.199233e-47,0.442500,sig,50.802339,46.921097,0.975968
Apoa1,2.589844,2.025723e-40,1.027379e-36,1.027379e-36,0.258984,sig,39.693420,35.988269,0.792002
Guca2b,2.380329,1.732794e-21,5.272893e-18,5.272893e-18,0.238033,sig,20.761253,17.277951,0.363411
Zg16,2.077089,2.553716e-21,6.475798e-18,6.475798e-18,0.207709,sig,20.592827,17.188707,0.448646


We can use `sc.pl.violin` to compare the gene expression between different condition.

In [8]:
celltypes_li=['TA']

sc.pl.violin(
    adata[adata.obs['cell_label'].isin(celltypes_li)],
    keys=['Reg3b','Reg3g','Apoa1'],
    groupby='condition'
)

## DEG with memento

memento is a python package for performing differential mean, variability, and correlation in single-cell RNA sequencing data.

Memento, an end-to-end method that implements a hierarchical model for estimating mean, residual variance, and gene correlation from scRNA-seq data and provides a statistical framework for hypothesis testing of these parameters.


In [9]:
# pip install memento-de
deg_obj=ov.single.DEG(
    adata,
    condition='condition',
    ctrl_group='Control',
    test_group='Salmonella',
    method='memento-de',
)
deg_obj.run(
    celltype_key='cell_label',
    celltype_group=['TA'],
    capture_rate=0.07, 
    num_cpus=12,
    num_boot=5000
)

✅ Differential expression analysis initialized
📊 DEG analysis using memento-de method
📊 Condition: condition, Control group: Control, Test group: Salmonella
📊 Celltype key: cell_label, Celltype group: ['TA']
Total cells: 533 will be used for DEG analysis


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done 404 tasks      | elapsed:    6.9s
[Parallel(n_jobs=12)]: Done 1404 tasks      | elapsed:   12.8s
[Parallel(n_jobs=12)]: Done 2804 tasks      | elapsed:   20.9s


✅ memento-de DEG analysis completed


[Parallel(n_jobs=12)]: Done 4544 out of 4544 | elapsed:   30.9s finished


In [10]:
res_memento=deg_obj.get_results()
res_memento.query('dv_coef > 1 & de_coef > 0').sort_values('dv_pval').head(5)

,gene,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval,baseMean
443,Btf3,stim,0.839043,0.231797,0.000236,2.935205,0.572195,3.023267e-07,0.133208
397,Birc5,stim,0.631178,0.185273,0.000661,1.198823,0.313596,8.280704e-05,0.000000
3421,Serinc2,stim,0.030935,0.238055,0.826321,2.286925,0.513116,1.613995e-04,0.133208
917,Dcaf8,stim,0.037631,0.316794,0.846642,1.833335,0.500870,2.680076e-04,0.000000
536,Ccnb2,stim,0.132247,0.234056,0.565606,1.126923,0.323989,3.477457e-04,0.005629


In [11]:
celltypes_li=['TA']

sc.pl.violin(
    adata[adata.obs['cell_label'].isin(celltypes_li)],
    keys=['Btf3','Serinc2','Birc5'],
    groupby='condition'
)

## DCT with scCODA

In omicverse, we only need one function `ov.single.DCT` to perfrom all DCT analysis tasks. We included `scCODA` and `milo` to perform the celltype abundance analysis.

Besides, you can also perform the analysis with `pertpy`'s api. `dct_obj.model` will be helpful.

More tutorial could be found in https://pertpy.readthedocs.io/en/stable/tutorials/notebooks/sccoda.html

In [12]:
dct_obj=ov.single.DCT(
    adata,
    condition='condition',
    ctrl_group='Control',
    test_group='Salmonella',
    cell_type_key='cell_label',
    method='sccoda',
    sample_key='batch',
)

✅ Differential cell type abundance analysis initialized
📊 DCT analysis using sccoda method
📊 Condition: condition, Control group: Control, Test group: Salmonella
• Automatic reference selection! Reference cell type set to Endocrine


No-U-turn HMC sampling is then initiated by calling sccoda_model.`run_nuts`().

We can use `help(dct_obj.model.run_nuts)` to obtain the argument as input.

In [14]:
dct_obj.run(
    num_samples=5000, #number of sampled values after burn-in.
    num_warmup=500, #Number of burn-in (warmup) samples.
)

sample: 100%|██████████| 5500/5500 [07:33<00:00, 12.14it/s, 127 steps of size 3.03e-02. acc. prob=0.72]  


✅ sccoda DCT analysis completed


In [15]:
res=dct_obj.get_results()
res.head()

Final Parameter  HDI 3%  HDI 97%  \
Covariate             Cell Type                                                 
conditionT.Salmonella Endocrine                     0.000000   0.000    0.000   
                      Enterocyte                    1.362798   0.840    1.831   
                      Enterocyte.Progenitor         0.000000  -0.333    0.654   
                      Goblet                        0.000000  -0.260    1.000   
                      Stem                          0.000000  -0.804    0.182   

                                                SD  Inclusion probability  \
Covariate             Cell Type                                             
conditionT.Salmonella Endocrine              0.000                 0.0000   
                      Enterocyte             0.261                 1.0000   
                      Enterocyte.Progenitor  0.155                 0.3216   
                      Goblet                 0.296                 0.4524   
                      Stem                   0.198                 0.3576   

                                             Expected Sample  log2-fold change  
Covariate             Cell Type                                                 
conditionT.Salmonella Endocrine                    26.064233         -0.490685  
                      Enterocyte                  323.552564          1.475416  
                      Enterocyte.Progenitor       100.440347         -0.490685  
                      Goblet                       43.796942         -0.490685  
                      Stem                        120.489666         -0.490685

### Adjusting the False discovery rate

scCODA selects credible effects based on their inclusion probability. The cutoff between credible and non-credible effects depends on the desired false discovery rate (FDR). A smaller FDR value will produce more conservative results, but might miss some effects, while a larger FDR value selects more effects at the cost of a larger number of false discoveries.

The desired FDR level can be easily set after inference via sim_results.set_fdr(). Per default, the value is 0.05, but we recommend to increase it up to 0.2 if no effects are found at a more conservative level.

In our example, setting a desired FDR of 0.4 reveals small effects on Endocrine and Tuft cells. Keep in mind that we chose this value only for instructive purposes, since there are no credible effects beside Enterocytes at lower FDR levels. In practice, expecting 40% of all credible effects to be false-positives is usually not recommended.

In [16]:
dct_obj.model.set_fdr(dct_obj.sccoda_data, 
                      modality_key="coda", 
                      est_fdr=0.6)
res=dct_obj.get_results()
res.sort_values('Final Parameter',ascending=False).head()

Final Parameter  HDI 3%  HDI 97%  \
Covariate             Cell Type                                                 
conditionT.Salmonella Enterocyte                    1.362798   0.840    1.831   
                      Goblet                        0.338849  -0.260    1.000   
                      Enterocyte.Progenitor         0.107106  -0.333    0.654   
                      TA.Early                      0.000541  -0.477    0.482   
                      Endocrine                     0.000000   0.000    0.000   

                                                SD  Inclusion probability  \
Covariate             Cell Type                                             
conditionT.Salmonella Enterocyte             0.261                 1.0000   
                      Goblet                 0.296                 0.4524   
                      Enterocyte.Progenitor  0.155                 0.3216   
                      TA.Early               0.133                 0.2712   
                      Endocrine              0.000                 0.0000   

                                             Expected Sample  log2-fold change  
Covariate             Cell Type                                                 
conditionT.Salmonella Enterocyte                  327.533634          1.493059  
                      Goblet                       62.217882          0.015814  
                      Enterocyte.Progenitor       113.170960         -0.318520  
                      TA.Early                    143.070037         -0.472261  
                      Endocrine                    26.384934         -0.473042

In [17]:
#ov.plot_set()
dct_obj.model.plot_boxplots(
    dct_obj.sccoda_data, 
    modality_key="coda", 
    feature_name="condition", 
    add_dots=True,
    figsize=(4,4),
    dpi=80,
)
ov.plt.show()

In [18]:
dct_obj.model.summary(
    dct_obj.sccoda_data, 
    modality_key="coda"
)

                                          Compositional Analysis summary                                           
┌──────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┐
│ Name                                         │ Value                                                            │
├──────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│ Data                                         │ Data: 6 samples, 8 cell types                                    │
│ Reference cell type                          │ Endocrine                                                        │
│ Formula                                      │ condition                                                        │
└──────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ cell_label                                                                                                      │
│ Endocrine                  1.196            36.623                                                              │
│ Enterocyte                 2.352           116.359                                                              │
│ Enterocyte.Progenitor      2.545           141.130                                                              │
│ Goblet                     1.715            61.540                                                              │
│ Stem                       2.727           169.301                                                              │
│ TA                         2.137            93.848                                                              │
│ TA.Early                   2.886           198.479                                                              │
│ Tuft                       0.470            17.720                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                              Final Parameter  Expected Sample  log2-fold change                 │
│ Covariate             Cell Type                                                                                 │
│ conditionT.Salmonella Endocrine                   0.000           26.385            -0.473                      │
│                       Enterocyte                  1.363          327.534             1.493                      │
│                       Enterocyte.Progenitor       0.107          113.171            -0.319                      │
│                       Goblet                      0.339           62.218             0.016                      │
│                       Stem                       -0.236           96.296            -0.814                      │
│                       TA                         -0.227           53.863            -0.801                      │
│                       TA.Early                    0.001          143.070            -0.472                      │
│                       Tuft                       -0.024           12.463            -0.508                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

## DCT with milo

Many biological conditions (disease, development, genetic KOs) can induce shifts in cell composition, where cells of a given state become enriched or depleted in response to a perturbation. With differential abundance analysis, we quantify consistent changes in cell composition across replicate samples. While differential abundance analysis can be performed on cell type clusters, it’s not always possible or practical to use precisely labeled clusters, especially when we are interested in studying transitional states, such as during developmental processes, or when we expect only a subpopulation of a cell type to be affected by the condition of interest. Milo is a method to detect compositional changes occurring in smaller subpopulations of cells, defined as neighbourhoods over the k-nearest neighbor (KNN) graph of cell-cell similarities.

Tutorials could be found in https://pertpy.readthedocs.io/en/stable/tutorials/notebooks/milo.html

### Build KNN graph

We can use omicverse functions to build a KNN graph. We set the dimensionality and value for k to use in subsequent steps.

Here the value of k indicates the smallest possible size of neighbourhood in which we will quantify differential abundance (i.e. with k=50 the smallest neighbourhood will have 50 cells). Depending on the number of samples, you might want to use a high value of k for neighbourhood analysis, to have sufficient power to estimate abundance fold-changes. Since here we have data from > 100 patients, we set k=150 to have on average more than one cell per donor in each neighbourhood.

In [19]:
ov.settings.cpu_gpu_mixed_init()

CPU-GPU mixed mode activated


In [20]:
adata=ov.pp.preprocess(adata,mode='shiftlog|pearson',n_HVGs=2000,
                       target_sum=50*1e4)
adata.raw = adata
adata = adata[:, adata.var.highly_variable_features]
ov.single.batch_correction(adata,batch_key='batch',
                                        methods='harmony',n_pcs=50)
ov.pp.neighbors(adata, n_neighbors=15, n_pcs=50,
               use_rep='X_harmony')
ov.pp.umap(adata)

Begin robust gene identification
After filtration, 14793/15215 genes are kept.     Among 14793 genes, 13948 genes are robust.
End of robust gene identification.
Begin size normalization: shiftlog and HVGs selection pearson
normalizing counts per cell
The following highly-expressed genes are not considered during normalization factor computation:
['Defa24', 'Fabp6', 'Gcg', 'Gip', 'Nts', 'Reg3b', 'Reg4', 'Sct', 'Spink4', 'Sst', 'Tff3', 'Zg16']
    finished (0:00:00)
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'highly_variable_nbatches', int vector (adata.var)
    'highly_variable_intersection', boolean vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'residual_variances', float vector (adata.var)
Time to analyze data in cpu: 1.0320501327514648 seconds.
End of size normalization: shiftlog and HVGs selection pearson
...Begin using h

2025-08-05 12:50:26,772 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-08-05 12:50:27,793 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-08-05 12:50:27,891 - harmonypy - INFO - Iteration 1 of 10
2025-08-05 12:50:29,225 - harmonypy - INFO - Iteration 2 of 10
2025-08-05 12:50:30,484 - harmonypy - INFO - Iteration 3 of 10
2025-08-05 12:50:31,740 - harmonypy - INFO - Iteration 4 of 10
2025-08-05 12:50:32,999 - harmonypy - INFO - Iteration 5 of 10
2025-08-05 12:50:33,850 - harmonypy - INFO - Iteration 6 of 10
2025-08-05 12:50:34,420 - harmonypy - INFO - Iteration 7 of 10
2025-08-05 12:50:34,820 - harmonypy - INFO - Iteration 8 of 10
2025-08-05 12:50:35,279 - harmonypy - INFO - Converged after 8 iterations


🖥️ Using Scanpy CPU to calculate neighbors...
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:03)
🔍 [2025-08-05 12:50:38] Running UMAP in 'cpu-gpu-mixed' mode...
🚀 Using torch GPU to calculate UMAP...
NVIDIA CUDA GPUs detected:
📊 [CUDA 0] NVIDIA GeForce RTX 2080 Ti
    ||||||||||||||||||||||||------ 9368/11264 MiB (83.2%)
📊 [CUDA 1] NVIDIA GeForce RTX 2080 Ti
    ------------------------------ 160/11264 MiB (1.4%)
computing UMAP🚀
    finished ✅: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:00:06)
✅ UMAP completed successfully.


In [21]:
ov.pl.embedding(
    adata,
    basis='X_umap',
    color=['batch','cell_label'],
)

### Differential abundance testing with GLM

Similar to the scCODA approach, we use `omicverse.single.DCT` to perform differential cell–abundance analysis, except that here we set the `method` to `milo`. In future releases, we will gradually remove the rpy2‐based edgeR dependency so that milo analyses can run entirely in a native Python environment.


In [22]:
dct_obj=ov.single.DCT(
    adata,
    condition='condition',
    ctrl_group='Control',
    test_group='Salmonella',
    cell_type_key='cell_label',
    method='milo',
    sample_key='batch',
    use_rep='X_harmony'
)

✅ Differential cell type abundance analysis initialized
📊 DCT analysis using milo method
📊 Condition: condition, Control group: Control, Test group: Salmonella
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)


In [23]:
dct_obj.run()

✅ milo DCT analysis completed


We can start inspecting the results of our DA analysis from a couple of standard diagnostic plots.

In [24]:
import matplotlib.pyplot as plt
old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = [6, 3]
plt.subplot(1, 2, 1)
plt.hist(dct_obj.mdata["milo"].var.PValue, bins=50)
plt.xlabel("P-Vals")
plt.subplot(1, 2, 2)
plt.plot(dct_obj.mdata["milo"].var.logFC, -ov.np.log10(dct_obj.mdata["milo"].var.SpatialFDR), ".")
plt.xlabel("log-Fold Change")
plt.ylabel("- log10(Spatial FDR)")
plt.tight_layout()
plt.rcParams["figure.figsize"] = old_figsize

We can see that for the majority of neighbourhoods, almost all cells have the same cell type label. We can rename neighbourhoods where less than 60% of the cells have the top label as “Mixed”

In [25]:
plt.hist(dct_obj.mdata["milo"].var["nhood_annotation_frac"], bins=30)
plt.xlabel("celltype fraction")

Text(0.5, 0, 'celltype fraction')

In [26]:
res=dct_obj.get_results(mix_threshold=0.6)
res.head()

,index_cell,kth_distance,logFC,logCPM,F,PValue,FDR,adjust.method,comparison,test,SpatialFDR,Nhood_size,nhood_annotation,nhood_annotation_frac
0,B1_AATAAGCTAGAGAT_Control_Enterocyte.Progenitor,11.100586,-0.813165,12.359344,7.826394,0.015007,0.045259,BH,-1*conditionControl 1*conditionSalmonella,glm,0.044600,331.0,Mixed,0.456193
1,B1_ACAAAGGATTGTGG_Control_Stem,9.838944,-0.749778,13.111821,8.497840,0.011964,0.040188,BH,-1*conditionControl 1*conditionSalmonella,glm,0.039194,551.0,Mixed,0.589837
2,B1_ACATGGTGCCGTTC_Control_Goblet,15.230800,0.143952,11.935857,0.118641,0.735988,0.776876,BH,-1*conditionControl 1*conditionSalmonella,glm,0.780154,241.0,Goblet,0.991701
3,B1_AGCGGCACCAGAAA_Control_Stem,9.446790,-0.696262,13.245266,6.665614,0.022656,0.058277,BH,-1*conditionControl 1*conditionSalmonella,glm,0.057199,604.0,Stem,0.798013
4,B1_ATTACCTGGGATTC_Control_TA,10.838922,-0.709047,12.384033,5.436002,0.036320,0.079148,BH,-1*conditionControl 1*conditionSalmonella,glm,0.078575,340.0,Mixed,0.444118


### Visualization

This is my favorite Milo plot. First, we create a `color_dict` to specify the cell colors we want to visualize.


In [27]:
color_dict=dict(zip(
    adata.obs['cell_label'].cat.categories,
    ov.pl.green_color[:4]+ov.pl.purple_color
))
color_dict['Mixed']='#c2c2c2'
fig, ax = plt.subplots(figsize=(3, 3))
ov.pl.embedding(
    adata,
    basis='X_umap',
    color=['cell_label'],
    palette=color_dict,
    ax=ax,
    #fig_size=(3,3)
)

In [28]:
#fig, ax = plt.subplots(figsize=(3, 4))
dct_obj.model.plot_da_beeswarm(
    dct_obj.mdata, 
    alpha=0.1,
    return_fig=True,
    palette=color_dict,
)
ov.plt.xticks(fontsize=12)
ov.plt.yticks(fontsize=12)
ov.plt.ylabel('T/NK Cells',fontsize=13)
ov.plt.text(-1,-0.75,'Enriched in\nControl',fontsize=13)
ov.plt.text(1,-0.75,'Enriched in\nSalmonella',fontsize=13)
#fig

Text(1, -0.75, 'Enriched in\nSalmonella')